# This jupyter notebook extracts metadata from the given pdf file.

### As a Pre-requisite to deal with pdf file, installing PyPDF4 library. Other popular library such as PyPDF, pdfminer and textract libraries are missing new lines for the given document which combines two words and also miss cell data in the pdf file which turns into important data loss.

In [1]:
import sys
!{sys.executable} -m pip install PyPDF4

In [2]:
import PyPDF4
import re
import io

### reads pdf file and converts pdf file to raw text data.

In [3]:
def  extract_text(file):
    pdfFileObj = open(file, 'rb')
    pdfReader = PyPDF4.PdfFileReader(pdfFileObj)

    text = []
    for pageNo in range(pdfReader.getNumPages()):
        pageObj = pdfReader.getPage(pageNo)
        pages_text  = pageObj.extractText()

        # spliting data to list of lines, so that it becaomes easy to find cell data.
        text = text + pages_text.split('\n')

    return text

### extracts applicant data from the plain text. To deal with blank cells using, filters to get data.

In [4]:
def extract_applDetail(data):
    flag = False
    apName = ''
    apAddress = ''
    apcompName = ''

    for k, v in enumerate(data):
        # limiting loop to find only specific metadata
        if v == "2. Applicant Details":
            flag = True
            continue

        if flag:
            # checking for first name cell data
            if v == "First Name" and data[k+1] != 'Surname':
                apName = apName + data[k+1]
                continue

            # checking for surname cell data
            if v == 'Surname' and data[k+1] != 'Company name':
                apName = apName + ' ' + data[k+1]
                continue

            # checking for company cell data
            if v == 'Company name' and data[k+1] != 'Address line 1':
                apcompName = apcompName + data[k+1]
                continue

            # cehcking for address line data and so on
            if v == 'Address line 1' and data[k+1] != 'Address line 2':
                apAddress = apAddress + data[k+1]
                continue
            if v == 'Address line 2' and data[k+1] != 'Address line 3':
                apAddress = apAddress + ' ' + data[k+1]
                continue
            if v == 'Address line 3' and data[k+1] != 'Town/city':
                apAddress = apAddress + ' ' + data[k+1]
                continue
            if v == 'Town/city' and data[k+1] != 'Country':
                apAddress = apAddress + ', ' + data[k+1]
                continue
            if v == 'Country' and not data[k+1].startswith('Planning Portal'):
                apAddress = apAddress + ', ' + data[k+1]

            # breaks the loop when address field completed
            if v == 'Postcode':
                break
            
    return apName, apcompName, apAddress

### extracts agent data from the plain text. Applying same filters like above.

In [5]:
def extract_agentDetail(data):
    flag = False
    agName = ''
    agAddress = ''
    agcompName = ''

    for k, v in enumerate(data):
        if v == "3. Agent Details":
            flag = True
            continue

        if flag:
            if v == "First name" and data[k+1] != 'Surname':
                agName = agName + data[k+1]
                continue
            if v == 'Surname' and data[k+1] != 'Company name':
                agName = agName + ' ' + data[k+1]
                continue
            if v == 'Company name' and data[k+1] != 'Address line 1':
                agcompName = agcompName + data[k+1]
                continue
            if v == 'Address line 1' and data[k+1] != 'Address line 2':
                agAddress = agAddress + data[k+1]
                continue
            if v == 'Address line 2' and data[k+1] != 'Address line 3':
                agAddress = agAddress + ' ' + data[k+1]
                continue
            if v == 'Address line 3' and data[k+1] != 'Town/city':
                agAddress = agAddress + ' ' + data[k+1]
                continue
            if v == 'Town/city' and data[k+1] != 'Country':
                agAddress = agAddress + ', ' + data[k+1]
                continue
            if v == 'Country' and data[k+1] != 'Postcode':
                agAddress = agAddress + ', ' + data[k+1]
            if v == 'Postcode':
                break
            
    return agName, agcompName, agAddress

### extracts proposed materials data.

In [6]:
def extract_proposed_materials(data):
    flag = False
    materials = []

    for k, v in enumerate(data):
        if v == '7. Materials':
            flag = True
            continue

        if flag:
            if v == 'Description of proposed materials and finishes:':
                materials.append(data[k+1])
                continue
            if v == '8. Pedestrian and Vehicle Access, Roads and Rights of Way':
                break
    
    return materials

In [7]:
text = extract_text('Assignment.pdf')
App_Name, App_CompName, App_Addresss = extract_applDetail(text)
Agent_Name, Agent_CompName, Agent_Addresss = extract_agentDetail(text)
materials = extract_proposed_materials(text)

In [8]:
print('Applicant Name: {0},\nApplicant Company Name: {1},\nApplicant Address: {2}'.format(App_Name, App_CompName, App_Addresss))

Applicant Name:  Amin,
Applicant Company Name: Britbuild Properties Ltd,
Applicant Address: 166 Weir Road, London, United Kingdom


In [9]:
print('Agent Name: {0},\nAgent Company Name: {1},\nAgent Address: {2}'.format(Agent_Name, Agent_CompName, Agent_Addresss))

Agent Name: Ellen Creegan,
Agent Company Name: Iceni Projects,
Agent Address: This is the Space 68 Quay Street, Manchester


In [10]:
print('Proposed Materials: ', materials)

Proposed Materials:  ['Stock brickwork', 'Zinc cladding', 'Aluminium clad timber glazed windows', 'Aluminium clad timber doors']
